In [5]:
import sys
sys.path.insert(0, "../") # Add root/ to the path
# from src.magicmath.magicmath import MagicMath
from src.magicmath.data_generator import DataFrameGenerator

In [18]:
# Generate Mock Data
data_generator = DataFrameGenerator()

columns = ["Market Cap", "EBIT", "Cash&Cash Equivalent", "Total Current Assets", "Property, Plant & Equipment", "Total Current Liabilities", "Total debt", "Depreciation"]
n_rows = 25
data = data_generator.generate_dataframe(columns, n_rows)
print(data.head())


   Market Cap      EBIT  Cash&Cash Equivalent  Total current Assets  \
0    0.296894  0.897736              0.602937              0.739034   
1    0.557888  0.983423              0.963939              0.220214   
2    0.230679  0.161388              0.926559              0.037566   
3    0.836528  0.154402              0.450196              0.025922   
4    0.821825  0.699526              0.036720              0.442539   

   Property, Plant & Equipment  Total Current Liabilities  Total debt  \
0                     0.161366                   0.273449    0.100001   
1                     0.118544                   0.989233    0.872953   
2                     0.758207                   0.197448    0.165198   
3                     0.774922                   0.265992    0.385645   
4                     0.602206                   0.377653    0.376237   

   Depreciation  
0      0.210316  
1      0.166446  
2      0.179934  
3      0.169200  
4      0.206679  


In [ ]:
#"ROIC (return on invested capital) % = EBIT / (Net Working Capital + Net Fixed Assets)
# EY (earnings yield) % = EBIT / Enterprise value"
# Calculate Net Working Capital , Net Fixed Assets , Enterprise Value				
					
					
